http request pollyfill?


In [ ]:
var bluebird = require('bluebird');
var req = bluebird.promisify(require('request'));

function request(...args) {
    return req(...args)
        .then(res => {
            if(res.statusCode < 200 || res.statusCode >= 300) {
                throw new Error('Error: ' + res.statusCode + ' - ' + JSON.stringify(res.body));
            }
            var isJson = (res.headers['content-type'] || '').match(/application\/json/ig);
            if(isJson && typeof res.body === 'string') {
                res.body = JSON.parse(res.body);
            }
            return res;
        })
}

module.exports = {
    request
};

